In [ ]:
import numpy as np
import psi4
from matplotlib import pyplot as plt

In [ ]:
def X_matrix(S):
    evalues,U= np.linalg.eigh(S)
    s=np.diag(1/np.sqrt(evalues))
    X=np.matmul(np.matmul(U,s),np.conjugate(U.T)) #X=S^(-1/2)  
    if verif_X_matrix(X,S):
        return(X)
    else: 
        print('problem with X')
        return
    
def verif_X_matrix(X,S):
    Xd=np.conjugate(X.T)
    verif=np.matmul(Xd,np.matmul(S,X)) #verif=X^dagger*S*X
    if ( abs(verif-np.identity(len(X)) )<(10**-10)).all():
        return(True)
    else:
        return(False)


In [ ]:

def G_matrix(P,v):
    n=len(P)
    G=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    G[i,j]+=P[k,l]*(v[i][j][k][l]-0.5*v[i][l][k][j])
    return(G)

def Ortho_fmatrix(F,X):
    FX=np.matmul(F,X)
    return(np.matmul(np.conjugate(X.T),FX))


In [ ]:
def coef_matrix(f,X):
    energy,c = np.linalg.eigh(f)
    C = np.matmul(X,c)
    return(C)


In [ ]:
def new_density_matrix(C,Bas_n,N):
    P=np.zeros_like(C)
    for i in range(Bas_n):
        for j in range(Bas_n):
            for a in range(int(N/2)):
                P[i,j]+=2*C[i,a]*C[j,a]
    return(P)


In [ ]:
#convergence de l'algo avec le commutateur

def convergence(F,P,S,lim):
    FPS=np.matmul(F,np.matmul(P,S))
    SPF=np.matmul(S,np.matmul(P,F))
    if (abs(FPS-SPF)<=lim).all():
        return True
    else:
        return False

In [ ]:
def SCF_energy(S,v,H,Bas_n,N,lim,max_iter,mol):
    #Initialisation
    X=X_matrix(S)
    P=np.zeros((Bas_n,Bas_n))
    New_Energy,Old_Energy=0,0
    Vnn=mol.nuclear_repulsion_energy()
    F=H
    #Boucle convergence
    for n_iter in range(max_iter):
        #Calcul de la nouvelle matrice densité
        f=Ortho_fmatrix(F,X)
        C=coef_matrix(f,X)
        P=new_density_matrix(C,Bas_n,N)
        #Matrice de Fock
        G=G_matrix(P,v)
        F=H+G
        #Calcul de l'énergie
        Old_Energy=New_Energy 
        New_Energy=0.5*np.trace(np.matmul(P,H+F))
        #Condition de convergence
        if abs(New_Energy-Old_Energy)<lim:
            return(New_Energy+Vnn)
    print('Did not converge')
    return(New_Energy+Vnn)


In [ ]:
def Diatomic_Energy(D,atome1,atome2,Basis_set):
    # géométrie de la molécule
    mol = psi4.geometry("""
    """+atome1+"""  0.0 0.0 0.0 
    """+atome2+"""  0.0 """+str(D)+""" 0.0

    """)
    #Paramètres de Psi4
    geo_unit = psi4.core.GeometryUnits(0) # Distance en Angstrom
    null=psi4.core.Molecule.set_units(mol,geo_unit) 
    null=psi4.core.Molecule.set_molecular_charge(mol,0) #Charge de la molécule
    null=psi4.core.Molecule.set_multiplicity(mol,1) #Multiplicité
    
    #Fonction d'onde de la molécule
    wf = psi4.core.Wavefunction.build(mol,basis=Basis_set)
    #Différentes intégrales calculées par psi4
    mints = psi4.core.MintsHelper(wf.basisset())
    S = np.asarray(mints.ao_overlap()) #Recouvrement
    v = np.asarray(mints.ao_eri()) #Intégrales biélectroniques
    T = np.asarray(mints.ao_kinetic()) 
    V = np.asarray(mints.ao_potential())
    H=T+V #Matrice Hamiltonienne
    
    Bas_n = S.shape[0] #Nombre d'atomiques orbitales utilisées
    N= int(2*wf.nalpha()) #Nombre d'électrons
    lim=10**(-8) #Critère de convergence
    max_iter=100 
    return(SCF_energy(S,v,H,Bas_n,N,lim,max_iter,mol))
    